In [10]:
pip install mlflow xgboost boto3 scikit-learn seaborn matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00


In [11]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

In [ ]:
import os

AWS_ACCESS_KEY_ID = input("s3-upload-user ")
AWS_SECRET_ACCESS_KEY = input("vrwV3P#= ")
AWS_REGION = "us-east-1"

In [16]:
import boto3

session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=AWS_REGION
)

s3 = session.client("s3")

print("New session created!")

New session created!


In [17]:
import pandas as pd
# Connect to S3
s3 = session.client("s3")

# Bucket and file
bucket_name = "my-unique-dataset-bucketm"
file_key = "AB_NYC_2019.csv"

# Load CSV into pandas
obj = s3.get_object(Bucket=bucket_name, Key=file_key)
df = pd.read_csv(obj["Body"])

df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [18]:
df = df.drop(columns=["id", "host_name", "host_id", "last_review"])
df["reviews_per_month"] = df["reviews_per_month"].fillna(0)
df = df[df["price"] < 1000]
df = df[df["minimum_nights"] < 365]


In [19]:
sns.set(style="whitegrid")

# 1. Price Distribution
plt.figure(figsize=(8,6))
sns.histplot(df["price"], bins=50, kde=True)
plt.title("Distribution of Airbnb Prices")
plt.savefig("eda_price_dist.png")
plt.close()

# 2. Room Type vs Price
plt.figure(figsize=(8,6))
sns.boxplot(x="room_type", y="price", data=df)
plt.title("Price vs Room Type")
plt.savefig("eda_roomtype.png")
plt.close()

# 3. Correlation Heatmap
plt.figure(figsize=(10,8))
corr = df.select_dtypes(exclude="object").corr()
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap")
plt.savefig("eda_heatmap.png")
plt.close()


In [20]:
y = df["price"]
X = df.drop(columns=["price"])

cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(exclude="object").columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [21]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols)
    ]
)

model = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("xgb", XGBRegressor(
            n_estimators=300,
            learning_rate=0.1,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        ))
    ]
)


In [23]:
import mlflow
import mlflow.xgboost
import mlflow.sklearn

In [24]:
mlflow.set_experiment("Price_Prediction")


/usr/local/lib/python3.12/dist-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/19 19:47:44 INFO mlflow.tracking.fluent: Experiment with name 'Price_Prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/106948029131727585', creation_time=1763581664527, experiment_id='106948029131727585', last_update_time=1763581664527, lifecycle_stage='active', name='Price_Prediction', tags={}>

In [25]:
with mlflow.start_run():

    # Enable autologging
    mlflow.xgboost.autolog()

    # Train model
    model.fit(X_train, y_train)

    # Predictions
    preds = model.predict(X_test)

    # Metrics
    mae = mean_absolute_error(y_test, preds)
    rmse = np.sqrt(mean_squared_error(y_test, preds))
    r2 = r2_score(y_test, preds)

    # Log metrics manually
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("RMSE", rmse)
    mlflow.log_metric("R2", r2)

    # Log model
    mlflow.sklearn.log_model(model, "airbnb_price_model")

    # Log EDA images as artifacts
    mlflow.log_artifact("eda_price_dist.png")
    mlflow.log_artifact("eda_roomtype.png")
    mlflow.log_artifact("eda_heatmap.png")

    print("Model Performance:")
    print("MAE :", mae)
    print("RMSE:", rmse)
    print("R²  :", r2)


2025/11/19 19:48:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 19:48:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/19 19:48:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model Performance:
MAE : 47.602272033691406
RMSE: 82.74233171135106
R²  : 0.44850367307662964
